In [2]:
import pypsa


In [5]:
import pypsa_rd


In [8]:
network = pypsa.Network()

In [14]:
wetwork = pypsa_rd.Network()

In [18]:
## Minimal 3-node example of PyPSA linear optimal power flow
#
#Available as a Jupyter notebook at <http://www.pypsa.org/examples/minimal_example_lopf.ipynb>.

import numpy as np

network = pypsa_rd.Network()

#add three buses
for i in range(3):
    network.add("Bus","My bus {}".format(i))

print(network.buses)

#add three lines in a ring
for i in range(3):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%3),
                x=0.0001,
                s_nom=60)

print(network.lines)

#add a generator at bus 0
network.add("Generator","My gen 0",
            bus="My bus 0",
            p_nom=100,
            marginal_cost=50)

#add a generator at bus 1
network.add("Generator","My gen 1",
            bus="My bus 1",
            p_nom=100,
            marginal_cost=25)

print(network.generators)

print(network.generators.p_set)

#add a load at bus 2
network.add("Load","My load",
            bus="My bus 2",
            p_set=100)

print(network.loads)

print(network.loads.p_set)

#Do a linear OPF

def my_f(network,snapshots):
    print(snapshots)


network.lopf(extra_functionality=my_f)

#Cheap generator 1 cannot be fully dispatched because of network constraints,
#so expensive generator 0 also has to dispatch
print(network.generators_t.p)

#network flows
print(network.lines_t.p0)

#Line 1 is congested
print(abs(network.lines_t.p0)/network.lines.s_nom)

#Power flows towards lower voltage angles
print(network.buses_t.v_ang*180/np.pi)

#In linear approximation, all voltage magnitudes are nominal, i.e. 1 per unit
print(network.buses_t.v_mag_pu)

#At bus 2 the price is set above any marginal generation costs in the model, because to dispatch to
#it from expensive generator 0, also some dispatch from cheap generator 1 has to be substituted from generator0
#to avoid overloading line 1.
print(network.buses_t.marginal_price)




INFO:pypsa_rd.opf:Performed preliminary steps
INFO:pypsa_rd.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa_rd.opf:Solving model using glpk


attribute  v_nom type    x    y carrier  v_mag_pu_set  v_mag_pu_min  \
My bus 0     1.0       0.0  0.0      AC           1.0           0.0   
My bus 1     1.0       0.0  0.0      AC           1.0           0.0   
My bus 2     1.0       0.0  0.0      AC           1.0           0.0   

attribute  v_mag_pu_max control sub_network  
My bus 0            inf      PQ              
My bus 1            inf      PQ              
My bus 2            inf      PQ              
attribute      bus0      bus1 type       x    r    g    b  s_nom  \
My line 0  My bus 0  My bus 1       0.0001  0.0  0.0  0.0   60.0   
My line 1  My bus 1  My bus 2       0.0001  0.0  0.0  0.0   60.0   
My line 2  My bus 2  My bus 0       0.0001  0.0  0.0  0.0   60.0   

attribute  s_nom_extendable  s_nom_min  ...  v_ang_min  v_ang_max  \
My line 0             False        0.0  ...       -inf        inf   
My line 1             False        0.0  ...       -inf        inf   
My line 2             False        0.0  ...       -

ApplicationError: No executable found for solver 'glpk'